# Setting

In [3]:
import numpy as np
import torch
from torch import nn
from torch import Tensor

class config:
    dataroot = 'data'
    workers = 2
    imageSize = 32
    nc = 1 # input image channels
    nz = 100 # size of the latent z vector
    ngf = 32
    ngpu = 1
    state_E = '../../saved_models/VAE_fmnist/netE_pixel.pth'#'../saved_models/models/fmnist/netE.pth'
    state_G = '../../saved_models/VAE_cifar10/netG_pixel.pth'#'../saved_models/models/fmnist/netG.pth'
    batch_size = 1
    train_batchsize = 1
    num_samples=20
    beta = 1
opt = config

np.random.seed(1234)
torch.manual_seed(1234)
torch.backends.cudnn.deterministic = True
torch.set_deterministic(True)

# DataLoader

In [4]:
import torch
import DCGAN_VAE_pixel as DVAE
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

netG = DVAE.DCGAN_G(opt.imageSize, opt.nz, opt.nc, opt.ngf, opt.ngpu)

state_G = torch.load(opt.state_G, map_location = device)
netG.load_state_dict(state_G)

netE = DVAE.Encoder(opt.imageSize, opt.nz, opt.nc, opt.ngf, opt.ngpu)
state_E = torch.load(opt.state_E, map_location = device)
netE.load_state_dict(state_E)

netG.to(device)
netE.to(device)
netE.eval()
netG.eval()

RuntimeError: Error(s) in loading state_dict for DCGAN_G:
	Missing key(s) in state_dict: "main.initial:100-128:convt.weight", "main.initial:128:batchnorm.weight", "main.initial:128:batchnorm.bias", "main.initial:128:batchnorm.running_mean", "main.initial:128:batchnorm.running_var", "main.pyramid:64-32:convt.weight", "main.pyramid:32:batchnorm.weight", "main.pyramid:32:batchnorm.bias", "main.pyramid:32:batchnorm.running_mean", "main.pyramid:32:batchnorm.running_var", "main.final:32-1:convt.weight". 
	Unexpected key(s) in state_dict: "main.initial:100-256:convt.weight", "main.initial:256:batchnorm.weight", "main.initial:256:batchnorm.bias", "main.initial:256:batchnorm.running_mean", "main.initial:256:batchnorm.running_var", "main.initial:256:batchnorm.num_batches_tracked", "main.pyramid:256-128:convt.weight", "main.pyramid:128:batchnorm.weight", "main.pyramid:128:batchnorm.bias", "main.pyramid:128:batchnorm.running_mean", "main.pyramid:128:batchnorm.running_var", "main.pyramid:128:batchn

In [3]:
netE.conv1

Conv2d(128, 100, kernel_size=(4, 4), stride=(1, 1))

fmnist

In [4]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision.datasets import ImageFolder

dataset_fmnist = dset.FashionMNIST(root=opt.dataroot, train=False, download=True, transform=transforms.Compose([
                                transforms.Resize((opt.imageSize)),
                                transforms.ToTensor(),]))
test_loader_fmnist = torch.utils.data.DataLoader(dataset_fmnist, batch_size=2,
                                            shuffle=False, num_workers=int(opt.workers))
    
dataset_mnist = dset.MNIST(root=opt.dataroot, train=False, download=True, transform=transforms.Compose([
                                transforms.Resize((opt.imageSize,opt.imageSize)),
                                transforms.ToTensor(),
                            ]))

test_loader_mnist = torch.utils.data.DataLoader(dataset_mnist, batch_size=opt.batch_size,
                                            shuffle=False, num_workers=int(opt.workers))

Calculate Fisher Score

In [5]:
dataset_fmnist_train = dset.FashionMNIST(root=opt.dataroot, train=True, download=True, transform=transforms.Compose([
                                transforms.Resize((opt.imageSize)),
                                transforms.ToTensor(),
                            ]))
dataloader_fmnist = torch.utils.data.DataLoader(dataset_fmnist_train, batch_size=opt.train_batchsize,
                                                shuffle=True, num_workers=int(opt.workers))

In [6]:
import matplotlib.pyplot as plt
from sklearn import metrics

def KL_div(mu,logvar,reduction = 'avg'):
    mu = mu.view(mu.size(0),mu.size(1))
    logvar = logvar.view(logvar.size(0), logvar.size(1))
    if reduction == 'sum':
        return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) 
    else:
        KL = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(),1) 
        return KL

def Calculate_fisher(dataloader, dicts, max_iter, num_samples):
    optimizer1 = optim.SGD(netE.parameters(), lr=0,momentum=0)
    optimizer2 = optim.SGD(netG.parameters(), lr=0,momentum=0)
    loss_fn = nn.CrossEntropyLoss(reduction = 'none')
    for i, (x, _) in enumerate(dataloader):

        optimizer1.zero_grad()
        optimizer2.zero_grad()
        x = x.repeat(num_samples,1,1,1).to(device)
        b = x.size(0)
        target = Variable(x.data.view(-1) * 255).long()
        [z,mu,logvar] = netE(x)
        kld = KL_div(mu,logvar)
        recon = netG(z)
        recon = recon.contiguous()
        recon = recon.view(-1,256)
        recl = loss_fn(recon, target)
        recl = torch.sum(recl) / b
        loss =  recl + kld.mean()
        grads = []
        loss.backward(retain_graph=True)
        
        for x in dicts:
            for param in x.parameters():
                grads.append(param.grad.view(-1)**2) 
        grads = torch.cat(grads)
        if i!=0:
            Grads = (i*Grads + grads)/(i+1)
            if i>max_iter:
                break
        else:
            Grads = grads

    Grads = torch.sqrt(Grads)
    Grads = Grads*(Grads>1e-3)
    Grads[Grads==0] = 1e-3
    normalize_factor = 2*np.sqrt(len(Grads))
    return Grads, normalize_factor

def Calculate_score(dataloader, dicts, Grads, normalize_factor, number, num_samples):
    optimizer1 = optim.SGD(netE.parameters(), lr=0,momentum=0)
    optimizer2 = optim.SGD(netG.parameters(), lr=0,momentum=0)
    loss_fn = nn.CrossEntropyLoss(reduction = 'none')
    score = []
    for i, (x, _) in enumerate(dataloader):
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        x = x.repeat(num_samples,1,1,1).to(device)
        b = x.size(0)
        target = Variable(x.data.view(-1) * 255).long()
        gradient_val=0
        [z,mu,logvar] = netE(x)
        kld = KL_div(mu,logvar)
        recon = netG(z)
        recon = recon.contiguous()
        recon = recon.view(-1,256)
        recl = loss_fn(recon, target)
        loss =  torch.sum(recl)/b + kld.mean()
        loss.backward(retain_graph=True)
        grads = []
        for x in dicts:
            for param in x.parameters():
                grads.append(param.grad.view(-1)) 
        grads = torch.cat(grads)
        gradient_val = torch.norm(grads/Grads)/normalize_factor
        score.append(gradient_val.detach().cpu())
        if i%number==number-1:
            break
    return score

def plot_hist(indist_Grads, outdist_Grads):
    indist_Grads = torch.tensor(indist_Grads)
    plt.hist(indist_Grads, bins=100)

    outdist_Grads = torch.tensor(outdist_Grads)
    plt.hist(outdist_Grads, bins=500, alpha=0.5)
    plt.xlim(0,10)


def AUROC(indist_Grads, outdist_Grads):    
    combined = np.concatenate((indist_Grads, outdist_Grads))
    label_1 = np.ones(len(indist_Grads))
    label_2 = np.zeros(len(outdist_Grads))
    label = np.concatenate((label_1, label_2))
    fpr, tpr, thresholds = metrics.roc_curve(label, combined, pos_label=0)
    #plot_roc_curve(fpr, tpr)
    rocauc = metrics.auc(fpr, tpr)
    print('AUC for Gradient Norm is: {}'.format(rocauc))
    plt.plot(fpr, tpr)

In [ ]:
dicts = [netG.main[-1],netE.conv1]
Grads, normalize_factor = Calculate_fisher(dataloader_fmnist, dicts, max_iter=10000, num_samples=opt.num_samples)

In [ ]:
fmnist_Gradients = Calculate_score(test_loader_fmnist, dicts, Grads, normalize_factor, 2000, opt.num_samples)

In [ ]:
mnist_Gradients = Calculate_score(test_loader_mnist, dicts, Grads, normalize_factor, 2000, opt.num_samples)

In [ ]:
plot_hist(fmnist_Gradients, mnist_Gradients)

In [ ]:
AUROC(fmnist_Gradients, mnist_Gradients)